# Guide of HOW TO use the Overkill API

This is a guide to help you use the Overkill API. The API is a RESTful API that allows you to interact with the Overkill platform. The API is designed to be simple and easy to use. This guide will walk you through the process of using the API to interact with the Overkill platform, using Python.

## GETTING STARTED

To get started, you will need to create an account on the Overkill platform. You can do this by visiting the Overkill website and signing up for an account. Once you have created an account, you will need to generate an API key. You can do this by visiting the API section of the Overkill website and generating an API key.

This account can also be registered using the API, but for this guide, we will assume that you have already created an account and generated an API key.

## AUTHENTICATION

To authenticate with the Overkill API, you will need to include your API keys in the headers of your requests. You can do this by including an `x-api-key` and `x-api-secret` header with the values. Here is an example of how you can authenticate with the Overkill API using Python:

In [2]:
# prepare the headers for the requests

import requests
import os
from pprint import pprint

url = 'https://api.princeofcrypto.com/v1/{}'

X_API_KEY = os.getenv('X_API_KEY', 'aaa')
X_API_SECRET = os.getenv('X_API_SECRET', 'aaa')


headers = {
    'x-api-key': X_API_KEY,
    'x-api-secret': X_API_SECRET,
    'Content-Type': 'application/json'
}

We will use following input data, for the endpoints, if needed:

```python
paramaters = {
    'param1': 'value1',
    'param2': 'value2'
}

post_data = {
    'key1': 'value1',
    'key2': 'value2'
}
```

and prepare full url with the endpoint type:

```python
url_get = url.format('coin/data')
url_post = url.format('coin/data')

response_get_endpoint = requests.get(url_get, headers=headers, params=paramaters)
response_post_endpoint = requests.post(url_post, headers=headers, json=post_data)
```

## Example of using the Overkill API

Here is an example of how you can use the Overkill API to get the data of a coin:

In [3]:
# OVERKILL DATA

from typing import Optional


def get_overkill_active_coins():
    url_get = url.format('coin/list')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    return response.json()

def get_overkill_coin_metadata(samples:int = 1):
    url_get = url.format(f'coin/metadata/daily?samples={samples}')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    return response.json()

def get_overkill_coin_data(coin_id: int, samples:int = 365):
    url_get = url.format(f'coin/data/daily?coin_id={coin_id}&samples={samples}')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    return response.json()

def get_telegram_filters():
    url_get = url.format('telegram/filters')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    telegram_filters = response.json().get('result')
    telegram_filters = [{k: v for k, v in obj.items() if k != '_id'} for obj in telegram_filters]

    url_get = url.format('telegram/expressions')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()   
    expressions = response.json().get('result')[0]

    for telegram_filter in telegram_filters:
        for key,val in expressions.items():
            if val == telegram_filter.get('expression'):
                telegram_filter['expression'] = key
                break
    return telegram_filters

def get_current_watchlist():
    url_get = url.format('coin/watch')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    return response.json()     

def get_signals(latest: int = 10, signal_type: str = "telegram-signal", username: Optional[str] = None):
    url_get = url.format('user/signal')
    params = {
        "latest": latest,
        "signal_type": signal_type,
        "username": username
    }
    response = requests.get(url_get, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

# COINGECKO DATA

def get_coingecko_coin_list():
    url_get = url.format('coin/coingecko/list')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    return response.json()


def get_coingecko_price_data(coin_id: int, samples:int = 365):
    url_get = url.format(f'coin/coingecko/data?coin_id={coin_id}&samples={samples}')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    return response.json()

def get_gecko_terminal_data(pool_address: str, network: str = "solana", timeframe: str = "minute"):
    # endpoint also accepts there params: from_timestamp: datetime, to_timestamp: datetime
    url_get = url.format('coin/gecko-terminal/data')
    params = {
        "pool_address": pool_address,
        "network": network,
        "timeframe": timeframe
    }
    response = requests.get(url_get, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

# YAHOO DATA

def get_yahoo_coin_list():
    url_get = url.format('coin/yahoo/list')
    response = requests.get(url_get, headers=headers)
    response.raise_for_status()
    return response.json()    

In [28]:
# get current telegram filters (only the ones that are to filter chat messages)

pprint(get_telegram_filters())

[{'expression': 'GreaterThanOrEqual',
  'filter_name': 'token_market_cap',
  'value': 3000},
 {'expression': 'LessThan', 'filter_name': 'token_market_cap', 'value': 400000},
 {'expression': 'GreaterThanOrEqual',
  'filter_name': 'token_liquidity',
  'value': 1000},
 {'expression': 'Equal', 'filter_name': 'token_burned', 'value': True},
 {'expression': 'GreaterThanOrEqual',
  'filter_name': 'token_burned_percent',
  'value': 50}]


In [68]:
# get current watchlist

pprint(get_current_watchlist())

# if result = [] -> no coins are on the watchlist right now

{'jsonrpc': '2.0', 'result': []}


All the endpoints are available here: https://api.princeofcrypto.com/redoc

## Examples

Below you can find usage examples, of the functions defined above.

In [ ]:
# get current active coins

active_coins = get_overkill_active_coins()
pprint(active_coins[0])

In [57]:
# get coin daily data

import json

coin_list_json = json.loads(active_coins[0]['body'])
coin_id_example = coin_list_json['result'][0]['coin_id']

coin_tohlcv = get_overkill_coin_data(coin_id_example).get('result') # time, open, high, low, close, volume
pprint(coin_tohlcv[0]) # first sample = 365 days ago

{'adj_close': '28091.568359',
 'close': '28091.568359',
 'date': '2023-05-01T00:00:00',
 'high': '29329.935547',
 'low': '27680.792969',
 'open': '29227.103516',
 'volume': '18655599976'}


In [44]:
pprint(coin_tohlcv[-1])

{'adj_close': '63217.136719',
 'close': '63217.136719',
 'date': '2024-04-29T00:00:00',
 'high': '63217.136719',
 'low': '63103.300781',
 'open': '63103.300781',
 'volume': '17391955968'}


In [ ]:
# coin metadata

pprint(get_overkill_coin_metadata().get('result'))

In [61]:
# coingecko coin list

pprint(get_coingecko_coin_list().get('result')[0])

{'_id': '01coin', 'name': '01coin', 'platforms': {}, 'symbol': 'zoc'}


In [63]:
# yahoo coin list

pprint(get_yahoo_coin_list().get('result')[0])

{'_id': 'ADF',
 'name': 'Art de Finance USD',
 'time_updated': '2024-04-21T17:15:40.182000'}


In [ ]:
# coingecko data

pprint(get_coingecko_price_data(coin_id_example))

Example signal:

```json
{
  "token_market_cap": 141010,
  "token_liquidity": 11900,
  "token_liquidity_supply": 0,
  "token_renounced": true,
  "token_burned": true,
  "token_burned_percent": 100,
  "token_network": "solana",
  "token_name": "McDolendz",
  "token_ticker": "MCDZ",
  "token_address": "EKZYsyQrvwdzPoPYcEULZ4Zx65PdaH3bCMiWpsjSrQiW",
  "token_supply": 0,
  "token_platform": "Raydium",
  "token_platform_address": "AaUaWBgSDn3ZupZDTLRU8GAyT5yrdfR1prGHdgv1axSp",
  "token_platform_price": 0,
  "token_decimals": 0,
  "created_at": {
    "$date": "2024-04-27T21:40:02.656Z"
  }
}
```

In [ ]:
# coingecko terminal price data (coins from the 'watch list' are saved in here - to calculate PROFIT/LOSS of the signals)

pprint(get_gecko_terminal_data("AaUaWBgSDn3ZupZDTLRU8GAyT5yrdfR1prGHdgv1axSp").get('result'))

In [ ]:
url_get = url.format('coin/data/daily?coin_id=1')
response = requests.get(url_get, headers=headers)
response.raise_for_status()
response.json()

## Send your signals

You can also send YOUR signals - will be saved & accessible LIVE to all websocket clients.


In [4]:
# Define required logic

from pydantic import BaseModel, Field
from typing import Optional, Union

class SignalMessage(BaseModel):
    message: Union[dict, str] = Field(..., description="Message")
    target_user_id: Optional[str] = Field(None, description="Target user id")

# SEND YOUR SIGNAL
def post_your_signal(signal: SignalMessage):
    url_get = url.format('user/signal')
    response = requests.post(url_get, headers=headers, json={**signal.model_dump()})
    response.raise_for_status()
    return response.json()   

In [10]:
my_signal = SignalMessage(
    message = {
        "token_market_cap": 141010,
        "token_address": "0xexample",
        "some_field": "BUY NOW!"
    }
)

my_message_signal = SignalMessage(
    message= "My public signal: BUY THIS NOW - TICHI GEM!",
)

res1 = post_your_signal(my_signal)
res2 = post_your_signal(my_message_signal)

pprint(res1)
pprint(res2)


{'jsonrpc': '2.0', 'result': 'ok'}
{'jsonrpc': '2.0', 'result': 'ok'}


In [6]:
my_message_signal = SignalMessage(
    message= "LOW CAP Watch list has been cleared automatically!",
)

pprint(post_your_signal(my_message_signal))

{'jsonrpc': '2.0', 'result': 'ok'}


On server we got (and saved to 'signals' collection):

```json
[{
  "_id": {
    "$oid": "662faf525d5257d23a41da87"
  },
  "username": "AlphaDragon",
  "timestamp": "2024-04-29 14:31:46",
  "message": {
    "token_market_cap": 141010,
    "token_address": "0xexample",
    "some_field": "BUY NOW!"
  }
  },
  {
  "_id": {
    "$oid": "662faf535d5257d23a41da88"
  },
  "username": "AlphaDragon",
  "timestamp": "2024-04-29 14:31:47",
  "message": "My public signal: BUY THIS NOW - TICHI GEM!"
  }
]
```

In [ ]:
# get signals
telegram_signal = get_signals()

pprint(telegram_signal)

In [11]:
# get user signals
user_signals = get_signals(signal_type="user-signal")

pprint(user_signals)

{'jsonrpc': '2.0',
 'result': [{'message': 'My public signal: BUY THIS NOW - TICHI GEM!',
             'signal_type': 'user-signal',
             'timestamp': '2024-04-29 14:31:47',
             'username': 'AlphaDragon'},
            {'message': {'some_field': 'BUY NOW!',
                         'token_address': 'example',
                         'token_market_cap': 141010},
             'signal_type': 'user-signal',
             'timestamp': '2024-04-29 14:31:46',
             'username': 'ErectorSpammer5000'}]}


In [12]:
# get specific user signals
user_signals = get_signals(signal_type="user-signal", username="AlphaDragon")

pprint(user_signals)

{'jsonrpc': '2.0',
 'result': [{'message': 'My public signal: BUY THIS NOW - TICHI GEM!',
             'signal_type': 'user-signal',
             'timestamp': '2024-04-29 14:31:47',
             'username': 'AlphaDragon'}]}
